In [1]:
import glob
import cv2 as cv
import json
import numpy as np
from collections import Counter
import operator
import nltk.data
import string
import re
from nltk.tokenize import TweetTokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk.tag.stanford import StanfordNERTagger
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from nltk.tag.stanford import StanfordPOSTagger
st_pos = StanfordPOSTagger('/root/nltk_data/classifiers/english-bidirectional-distsim.tagger', '/root/nltk_data/jars/stanford-postagger.jar')
st_ner = StanfordNERTagger('/root/nltk_data/classifiers/english.conll.4class.distsim.crf.ser.gz',
               '/root/nltk_data/jars/stanford-ner.jar') 
import pke
import pickle
from tqdm import tqdm

In [2]:
reports = []
for file in glob.glob("../data/fms/complaints_*.json"):
    print(file)
    for line in open(file, "r"):
        try:
            reports.append(json.loads(line))
        except:
            print('error')

../data/fms/complaints_2018-05-09.json
../data/fms/complaints_2018-05-07.json
../data/fms/complaints_2018-05-14.json
../data/fms/complaints_2018-05-15.json


In [3]:
_descriptions = []
for report in reports:
    if(report['category']=="Street cleaning"):
        _descriptions.append(report['descroption'])

_words = []

for des in _descriptions:
    for s in des:
        if(s != None):
            for w in s.split():
                _words.append(w.lower())
count_words = Counter(_words).most_common()     
count_word_dict = dict((w, c) for w, c in count_words)

In [4]:
unwanted_words = ['help', 'thanks', 'thank', 'please', 'months', 'month', 'weeks', 'week', 'council', 'years', 'year', 'days', 'day', 'asap', 'avenue', 'way',
                  'junction', 'people', 'peoples', 'someone', 'children', 'entrance', 'election', 'bridge', 'residents', 'drive', 'resident', 'many', 'much', 'enclosed'
                 , 'today', 'something', 'number', 'first', 'photos', 'photo', 'images', 'image', 'location','one', 'pictures', 'picture', 'night',
                 'could', 'one', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
location_words = ['road', 'rd', 'street', 'lane', 'way', 'highway', 'drive']
unwanted_ner = ['PERSON', 'ORGANIZATION', 'LOCATION']
unwanted_pos = ['VB']

In [5]:
f = open("keyphrase_pke/keyphrases/Street cleaning.extractor", "rb")
extractor = pickle.load(f)
keyphrase = [key for key, val in extractor.get_n_best(n=500, stemming=False)]

In [6]:
new_keyphrase = []
for keyp in keyphrase:
    val = min([count_word_dict[w] if w in count_word_dict else np.inf for w in keyp.split()])
    if(val>10):
        new_keyphrase.append(keyp)
keyphrase = new_keyphrase

In [7]:
f1 = open("keyphrase_pke/keyphrases/filtered_Street cleaning", "wb")
f2 = open("keyphrase_pke/keyphrases/removed_Street cleaning", "wb")

In [8]:
filtered_keyphrases = []
removed_keyphrases = []
for keyp in tqdm(keyphrase):
    sent = keyp
    flag = 0
    if(len(keyp.split())>3):
        flag = 1
    else:
        pos_tags = [pos for key, pos in st_pos.tag(sent.split())]
        ner_tags = [ner for key, ner in st_ner.tag(sent.upper().split())]
        for w in keyp.split():
            if(w in unwanted_words):
                flag = 1
                break
            if(w in location_words):
                for ner in ner_tags:
                    if ner in unwanted_ner:
                        flag = 2
                        break
                        
        for pos in pos_tags:
            if pos in unwanted_pos:
                flag = 3
                break
    
    if(flag>0):
        removed_keyphrases.append([keyp, flag])
    else:
        filtered_keyphrases.append(keyp)

  0%|          | 2/419 [00:07<26:08,  3.76s/it]/root/env/lib/python3.5/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 419/419 [25:59<00:00,  3.72s/it]


In [9]:
pickle.dump(filtered_keyphrases, f1)
pickle.dump(removed_keyphrases, f2)

In [10]:
keyphrase

['road',
 'litter',
 'pavement',
 'rubbish',
 'lots',
 'street bin',
 'please',
 'thanks',
 'side',
 'broken glass',
 'leaves',
 'week',
 'gutter',
 'problems',
 'full',
 'junction',
 'council',
 'drains',
 'smashed glass',
 'footpath',
 'months',
 'people',
 'cars',
 'area clean',
 'corner',
 'path',
 'time',
 'bags',
 'large amount',
 'children',
 'years',
 'weeds',
 'dangerous',
 'bottles',
 'someone',
 'excess tree debris',
 'day',
 'need',
 'mess',
 'end',
 'grass verge',
 'house',
 'mud',
 'bushes',
 'empty cans',
 'grass',
 'tree waste',
 'good thorough sweeping please',
 'parked cars',
 'street cleaning',
 'possible',
 'number',
 'dogs',
 'entrance',
 'chip shop',
 'school',
 'top end',
 'garden',
 'litter pick',
 'bus stop',
 'pile',
 'local residents',
 'amount',
 'kings park drive',
 'litter bins',
 'front',
 'dog mess',
 'part',
 'load',
 'length',
 'due',
 'roundabout',
 'road sweeper',
 'young street cleaner',
 'clear',
 'kerb gutters',
 'shelter',
 'least',
 'bad',
 'bro